In [8]:
from datasets import load_dataset

eli5= load_dataset("eli5", split="train_asks[:5000]") 

Found cached dataset eli5 (C:/Users/AIXI/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [9]:
eli5= eli5.train_test_split(test_size=0.2)

In [10]:
eli5["test"][22]

{'q_id': '2fhvmo',
 'title': 'Can somebody please explain the difference between a shear thinning and a thixotropic fluid?',
 'selftext': "As far as i'm aware a shear thinning fluid is one that shows a decrease in viscosity with increasing shear stress.\n\nHaving looked up the definition of a thixotropic fluid, it seems as though they're the same thing, although from what I've read is seems as though they do a have a subtly different meaning.\n\nCould anyone shed some light on what the difference between these two things is? Thanks.",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['ck9sk4e'],
  'text': ["I think your hangup is not so much that they are very similar, but rather that they are describing **ideal** behaviors that don't really exist independently from each other in real life materials that we know of.\n\nShear-thinning/thickening is just a [nonlinear relationship between shear stress and shear strain](_URL_1_); thins when sheared. Simple enough. Thixotro

In [11]:
# eli5= eli5.flatten()
# eli5["train"][22]

In [32]:
# for i in eli5["train"]:
#     print(i)

In [33]:
# import pandas as pd

def process_dataset(datasett):
    # list to store extracted elements
    titles = []
    selftexts = []
    texts = []

    for item in datasett:
       title = item['title']
       selftext = item['selftext']
       text = item['answers']['text'][0]

       titles.append(title)
       selftexts.append(selftext)
       texts.append(text)

    df = pd.DataFrame({
       'Title': titles,
       'Selftext': selftexts,
       'Text': texts
    })

    return df

In [34]:
df_train= process_dataset(eli5['train'])
df_test= process_dataset(eli5['test'])

In [35]:
print(len(df_train))
print(len(df_test))

4000
1000


In [37]:
df_test.head()

,Title,Selftext,Text
0,[Earth Sciences] What are the geological speci...,My searching of the internet and reddit hasn't...,It's tough to say the exact method of formatio...
1,How come different allotropes of an element ha...,"I have never studied chemistry, but I know tha...",The carbon atoms in diamond and graphite bond ...
2,Is there a scientific explanation for people w...,After digging through /r/deadbedrooms and /r/l...,Although this is probably incredibly more comp...
3,Ligo gravitational observatory and nuclear bombs?,Could a nuclear bomb trigger ligo? Would we kn...,Yes it would trigger ligo but they back check ...
4,Different ethnicity and their comfort levels i...,I have a diverse group of friends. Trinidadia...,Why would it have to be genetic? Maybe they ju...


In [50]:
def column_to_txt(dataframe, column_name, file_name):
    try:
        # get the specified column as a series
        column_data= dataframe[column_name]

        # write column values to txt file
        with open(file_name, 'w', encoding='utf-8') as file:
            for value in column_data:
                file.write(str(value)+ '\n')
        
        print(f'{column_name} values saved to {file_name}')
    except KeyError:
        print(f'column {column_name} not found in the dataframe')

column_to_txt(df_train, "Text", 'train.txt')
column_to_txt(df_test, "Text", 'test.txt')

Text values saved to train.txt
Text values saved to test.txt


In [48]:
# ## preprocessing of EVAL dataset
# eval_text_data= df_test['Text'].tolist()
# tokenized_eval_data= tokenizer(eval_text_data, truncation=True, padding=True)

# # creating a TextDataset for evaluation
# eval_dataset= TextDataset(tokenizer=tokenizer, file_path=None, column_names= ['input_ids', 'attention_mask'], data=tokenized_eval_data)

In [51]:
# import torch
# import pandas as pd
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
# from datasets import load_dataset

# # Initialize the GPT-2 tokenizer and model
# model_name = 'distilgpt2'
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)

# # Add a padding token to the tokenizer
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# # Tokenize your text data
# text_data = df_train['Text'].tolist()
# tokenized_data = tokenizer(text_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# # Tokenize evaluation data
# eval_text_data = df_test['Text'].tolist()
# tokenized_eval_data = tokenizer(eval_text_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# # Create datasets using the datasets library
# train_dataset = {'input_ids': tokenized_data['input_ids'], 'attention_mask': tokenized_data['attention_mask']}
# eval_dataset = {'input_ids': tokenized_eval_data['input_ids'], 'attention_mask': tokenized_eval_data['attention_mask']}

# # Create a DataCollator for Language Modeling
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [53]:
import datasets
# Initialize the GPT-2 tokenizer and model
model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load your text data from .txt files
with open('train.txt', 'r', encoding='utf-8') as train_file:
    train_data = train_file.readlines()

with open('test.txt', 'r', encoding='utf-8') as test_file:
    test_data = test_file.readlines()

# Tokenize your text data
tokenized_data = tokenizer(train_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# Tokenize evaluation data
tokenized_eval_data = tokenizer(test_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# Create datasets using the datasets library
train_dataset = datasets.Dataset.from_dict({
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask']
})

eval_dataset = datasets.Dataset.from_dict({
    'input_ids': tokenized_eval_data['input_ids'],
    'attention_mask': tokenized_eval_data['attention_mask']
})

# Create a DataCollator for Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


loading file vocab.json from cache at C:\Users\AIXI/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\vocab.json
loading file merges.txt from cache at C:\Users\AIXI/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\AIXI/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 

In [54]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Add the evaluation dataset here
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 21985
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8247
  Number of trainable parameters = 81912576


  0%|          | 0/8247 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\AIXI\AppData\Local\Temp\ipykernel_8380\3680148005.py:20 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\AIXI\\AppData\\Local\\Temp\\ipykernel_8380\\3680148005.py'                           │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:1527 in train      │
│                                                                                                  │
│   1524 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1525 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1526 │   │   )                                                                                 │
│ ❱ 1527 │   │   return inner_training_loop(                                                       │
│   1528 │   │   │   args=args,                                                                    │
│   1529 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1530 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:1775 in            │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1772 │   │   │   │   │   with model.no_sync():                                                 │
│   1773 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1774 │   │   │   │   else:                                                                     │
│ ❱ 1775 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1776 │   │   │   │                                                                             │
│   1777 │   │   │   │   if (                                                                      │
│   1778 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:2523 in            │
│ training_step                                                                                    │
│                                                                                                  │
│   2520 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2521 │   │                                                                                     │
│   2522 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2523 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2524 │   │                                                                                     │
│   2525 │   │   if self.args.n_gpu > 1:                                                           │
│   2526 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:2555 in            │
│ compute_loss                                               

In [42]:
torch.cuda.is_available()

True